In [6]:
# Step 1: Imports and setup
import polars as pl
from ib_insync import *

import asyncio

# Ensure the event loop integration works in notebook
util.startLoop()

# Step 2: Connect asynchronously to TWS or IB Gateway
ib = IB()
# await ib.connectAsync('127.0.0.1', 7496, clientId=6841)
await ib.connectAsync('127.0.0.1', 7497, clientId=6841)

ib.reqMarketDataType(3) 

In [7]:

symbol = 'JNJ'
underlying = Stock(symbol, exchange='SMART', currency='USD')

In [8]:
underlying

Stock(symbol='JNJ', exchange='SMART', currency='USD')

In [9]:
await ib.qualifyContractsAsync(underlying)


[Stock(conId=8719, symbol='JNJ', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='JNJ', tradingClass='JNJ')]

In [ ]:
# Step 4: Request option chain definition
chains = await ib.reqSecDefOptParamsAsync(
    underlying.symbol,
    '',  # exchange is empty string per IB API requirement
    underlying.secType,
    underlying.conId
)


In [ ]:

# Step 1: Define contracts
selected_strikes = [160.0, 170.0, 180.0]
expiry = '20260116'
symbol = underlying.symbol

options = [
    Option(
        symbol=symbol,
        lastTradeDateOrContractMonth=expiry,
        strike=strike,
        right='C',  # use 'C' not 'CALL'
        exchange='SMART'
    )
    for strike in selected_strikes
]

# Step 2: Qualify contracts
await ib.qualifyContractsAsync(*options)


[Option(conId=653218007, symbol='JNJ', lastTradeDateOrContractMonth='20260116', strike=160.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='JNJ   260116C00160000', tradingClass='JNJ'),
 Option(conId=653218142, symbol='JNJ', lastTradeDateOrContractMonth='20260116', strike=170.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='JNJ   260116C00170000', tradingClass='JNJ'),
 Option(conId=653218302, symbol='JNJ', lastTradeDateOrContractMonth='20260116', strike=180.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='JNJ   260116C00180000', tradingClass='JNJ')]

In [2]:
tickers = [ib.reqMktData(opt, snapshot=False) for opt in options]
await asyncio.sleep(2)  # Give it time to populate quotes


NameError: name 'options' is not defined

In [3]:
[ib.reqTickers(opt) for opt in options]


NameError: name 'options' is not defined

In [4]:
tickers

NameError: name 'tickers' is not defined

In [5]:

df = pl.DataFrame([{
    "symbol": opt.localSymbol,
    "strike": opt.strike,
    "expiry": opt.lastTradeDateOrContractMonth,
    "bid": tick.bid,
    "ask": tick.ask,
    "last": tick.last,
    "volume": tick.volume,
    "iv": tick.modelGreeks.impliedVol if tick.modelGreeks else None
} for opt, tick in zip(options, tickers)])


NameError: name 'options' is not defined

In [11]:
df

symbol,strike,expiry,bid,ask,last,volume,iv
str,f64,str,f64,f64,f64,f64,f64
"""JNJ 260116C00160000""",160.0,"""20260116""",13.35,14.05,-1.0,0.0,null
"""JNJ 260116C00170000""",170.0,"""20260116""",7.25,7.65,7.65,16.0,0.15996
"""JNJ 260116C00180000""",180.0,"""20260116""",3.3,3.45,3.4,21.0,0.150612


In [15]:
res = ib.reqMktData(underlying, snapshot=True) 

In [16]:
underlying

Stock(conId=8719, symbol='JNJ', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='JNJ', tradingClass='JNJ')

In [17]:
data = []
for option, ticker in zip(options, tickers):
    data.append({
        "symbol": option.localSymbol,
        "strike": option.strike,
        "expiry": option.lastTradeDateOrContractMonth,
        "right": option.right,  # 'C' or 'P'
        "bid": ticker.bid,
        "ask": ticker.ask,
        "last": ticker.last,
        "volume": ticker.volume,
        "impliedVol": ticker.modelGreeks.impliedVol if ticker.modelGreeks else None,
        "delta": ticker.modelGreeks.delta if ticker.modelGreeks else None,
        "gamma": ticker.modelGreeks.gamma if ticker.modelGreeks else None,
        "theta": ticker.modelGreeks.theta if ticker.modelGreeks else None,
        "vega": ticker.modelGreeks.vega if ticker.modelGreeks else None,
    })

In [18]:
opt = Option(symbol='AAPL', lastTradeDateOrContractMonth='20260116', strike=200, right='C', exchange='SMART')
await ib.qualifyContractsAsync(opt)
ticker = ib.reqMktData(opt, snapshot=False)
await asyncio.sleep(2)
print(ticker.bid, ticker.ask)

nan nan


In [16]:
ticker

Ticker(contract=Option(conId=653073465, symbol='AAPL', lastTradeDateOrContractMonth='20260116', strike=200.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='AAPL  260116C00200000', tradingClass='AAPL'))